# Memoria técnica Data Science

1. [Introducción](#introducción)
2. [Problemática](#problemática)
3. [Propuesta](#propuesta)
4. [BBDD SQL Relacional (PostgreSQL)](#bbdd-sql-relacional-postgresql)
5. [Automatismos](#automatismos)
   - 5.1 [Web Scraping (Selenium)](#web-scraping-selenium)
   - 5.2 [Tratamiento de imágenes y PDF (OpenCV, PyMuPDF, Regex)](#scrapeo-de-pdf-e-imágenes-opencv-pymupdf-regex)
6. [Modelo de Machine Learning](#modelo-de-machine-learning)
7. [Reporte Analítico](#reporte-analítico)
8. [Librerías](#librerias)

## 1. Introducción
Esta memoria representa el guión principal e hilo conductor del trabajo realizado por los compañeros de Data Science 2309 del grupo 3 del desafío de tripulaciones Javier de Alcazar, Maria Neches, Hugo Martín y Alejandro Campos, siendo tan solo una de las piezas que encajan en la propuesta integrada diseñada para solventar la necesidad y la propuesta realizada por Several Energy, la cual propone una actualización y mejoría de su sistema y flujo internos de trabajo. 

## 2.Análisis detallado del problema.

Se nos presenta la necesidad de crear una WebApp que sustituya, y amplie, las funciones de una tabla de excel.
Las deficiencias de la manera de trabajo actual son varias:

+ Mala escalabilidad. La empresa ha comenzado a tener problemas a la hora de usarla en diferentes situaciones.
+ Poco control del trabajo realizado. Enfarragosa manera de guardar historiales.
+ Exceso de trabajo por parte del asesor a la hora de introducir datos, provocando un largo tiempo de producción en la generación de propuestas.


## 3.Propuesta de solución tecnológica

La tarea de mejorar el trabajo realizado por una hoja excell con una WebApp plantea la oportunidad de usar una BBDD relacional, que acabe con los problemas de escalabilidad y registro de históricos. A ello añadimos la posibilidad de obtener datos de forma automatizada, aliviando la carga de trabajo del asesor.

+ BBDD SQL Relacional (PostgreSQL)
+ Webscrapping (Selenium)
+ Scrapeo de PDF e imágenes (OpenCV, PyMuPDF, Regex)
+ Modelo de predicción 
+ Dashboard (PowerBi)


## 4.BBDD SQL

Una vez analizadas las necesidades de la WebApp para solucionar los problemas actuales, se diseña la BBDD que cumpla las funciones necesarias para el proyecto.
Se trata de una BBDD relacional en PostreSQL 15.0, preparada para alojarse en la nube Azure. Se realizan los planos necesarios para el manejo de la misma un modelo lógico y un diagrama entidad relación conteniendo 12 entidades y 5 relaciones. [Link a la query de creación](/database/sql/create_tables.sql)

### Estructura
Esquema de la BBDD.

<img src="../aux_temp/Esquema.png" width="1900" />

[Link para ampliar](https://www.canva.com/design/DAF5e_nv_Bk/PNXGmx8l0Xajcoh2LPONFQ/edit?utm_content=DAF5e_nv_Bk&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton)

### Relación
Diagrama de relación.

<img src="../aux_temp/Diagrama.png" width="1900" />

[Link para ampliar](https://www.canva.com/design/DAFy6dl3Pe4/yYekcQiBpDDn7MCyx_-qiw/edit?utm_content=DAFy6dl3Pe4&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton)

# 5. Automatismos

El flujo del trabajo manual del asesor actual tiene dos tediosas tareas en tiempo que son candidatas a ser sustituidas por automatismos.
Se trata, por un lado,de la obtención de datos desde una intranet a la que se accede vía web, y con un código "CUPs" se extraen datos. Y por otro lado, la lectura personal de facturas para obtener datos de consumo y precios.

#### 5.1WebScraping

Mediante el uso de la librería Selenium, creamos un script que accede a la web de Candela energía para obtener la información necesaria.

In [ ]:
def webscraping_chrome_candelas(cups):

    path_driver = os.getcwd() + "\..\webscraping\chromedriver-win64\chromedriver.exe"
    # Create driver Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")

    servicio = Service(path_driver)
    driver = webdriver.Chrome(service=servicio, options=chrome_options)
    driver.get(URL)
    assert "Candela"
    time.sleep(3)

    # Login in candelas web

    driver.find_element(By.ID, "select_1").click()
    time.sleep(1)
    driver.find_element(By.ID, "select_option_3").click()
    driver.find_element(By.NAME, "usuario").send_keys(USER)
    driver.find_element(By.NAME, "password").send_keys(PASSWORD)
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/form/button').click()
    time.sleep(3)

    # Download info

    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/div[1]/div[1]/ul/li[3]/a').click()
    time.sleep(1)
    driver.find_element(By.ID, "input_6").send_keys(cups)
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[2]/md-tabs/md-tabs-content-wrapper/md-tab-content/div[1]/md-card/div[1]/form/div[4]/button').click()
    time.sleep(10)
    driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[2]/md-tabs/md-tabs-content-wrapper/md-tab-content/div[1]/md-content/md-card/md-toolbar/div[1]/button[1]').click()

    info = get_soup_info(driver)

    # Close driver
    driver.quit()

    return info

Esta funcion genera un retorno en formato json con la tarifa y los consumos y potencia anuales a las cuales podemos acceder mediante las credenciales de several energy en la web de candela energía

In [ ]:
{'rate': '2.0TD',
 'anual_consumption': '2.103 KWh',
 'anual_consumption_p1': '502 KWh',
 'anual_consumption_p2': '472 KWh',
 'anual_consumption_p3': '1129 KWh',
 'anual_consumption_p4': '0 KWh',
 'anual_consumption_p5': '0 KWh',
 'anual_consumption_p6': '0 KWh',
 'anual_power_p1': '4,6',
 'anual_power_p2': '4,6',
 'anual_power_p3': '',
 'anual_power_p4': '',
 'anual_power_p5': '',
 'anual_power_p6': ''}

## Tratamiento de imágenes y PDF

La extracción y autorellenado de datos se afianzó como parte clave de la mejora en el proceso de creación de propuestas. Tras el estudio con cliente sabemos que las facturas no les llegan todas en el mismo formato y de una manera standar si no que pueden ser desde PDF's hasta capturas de pantalla enviadas por whatsapp o fotos a documentos o pantallas.

Esto nos suponía un reto y la realización de 2 maneras de tratar esto. La idea es que tanto de los PDF's (de manera más sencilla) como en formato imagen JPG o PNG se extraiga todo el texto a texto plano y con regex se extraiga la información de los campos concretos.

Se realizaron un repositorio de diferentes funciones con varias librerias para ver de que manera podíamos extraer mejor los datos. Destacable el uso de la libreria langchain y la API de Open AI para usarlo como intérprete de las facturas, finalmente fue desestimado por que con regex se podría extraer los datos de igual o mejor manera y reduciamos los costes y el tiempo, al evitar una llamada a una API y estar haciendose en "local".

Tres son las tareas a realizar en este apartado.

1. Conversión de PDF a texto
2. Conversión de Imagen a texto
3. Scrapeo de Texto


In [ ]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

p_counter_kW=0
p_counter_kWh=0

def create_qa_chain(): 
    llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY)
    qa_chain = load_qa_chain(llm, chain_type="map_reduce")
    qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

    return qa_document_chain

In [ ]:
def extract_text_from_pdf(pdf):
    """
    Extract text content from a PDF file.

    Args:
        pdf (file-like object): PDF file object.

    Returns:
        str: Extracted text content from the PDF.
    """
    try:
        doc = fitz.open(stream=pdf.read(), filetype="pdf")
        full_text = ""

        for page_num in range(doc.page_count - 1):
            page = doc[page_num]
            page_text = page.get_text()
            full_text += page_text

        with open('data/pdf/invoice.pdf', 'wb') as output_pdf:
            pdf.seek(0)
            output_pdf.write(pdf.read())

        doc.close()

        return full_text
    except Exception as e:
        return str(e)

In [ ]:
def save_text_to_txt(text, txt_path):
    """
    Save text content to a text file.

    Args:
        text (str): Text content to be saved.
        txt_path (str): Path to the text file.

    Returns:
        None
    """
    if os.path.exists(os.path.dirname(txt_path)):
        try:
            with open(txt_path, 'w', encoding='utf-8') as txt_file:
                txt_file.write(text)
            print(f'Archivo guardado exitosamente en: {txt_path}')
        except Exception as e:
            print(f'Error al guardar el archivo: {e}')
    else:
        print(f'La ruta especificada no existe: {os.path.dirname(txt_path)}')

In [ ]:
def response_question_langchain(qa_document_chain, question):
    """
    Get responses to a question from a question-answering document chain.

    Args:
        qa_document_chain (AnalyzeDocumentChain): Question-answering document chain.
        question (str): The question to be answered.

    Returns:
        dict: A dictionary containing the question, responses, and any errors.
    """
    fragment_size = 4096
    all_responses= {"question": [],"response" : [], "error": []}
    with open("data/txt/invoice.txt", 'r', encoding='utf-8') as file:
        while True:
            part = file.read(fragment_size)
            if not part:
                break
            try:
                response = qa_document_chain.run(
                    input_document=part,
                    question=question,
                )
                all_responses["question"].append(question)
                all_responses["response"].append(response)
            except Exception as e:
                all_responses["error"].append(str(e))
    all_responses["question"] = all_responses["question"][0]
    return all_responses

In [ ]:
def invoice_clean_data(response):
    """
    Clean and process responses obtained from a question-answering task.

    Args:
        response (dict): Original responses containing question, responses, and errors.

    Returns:
        dict: Cleaned responses with irrelevant answers replaced and numeric values extracted.
    """
    clean_response = copy.deepcopy(response)
    float_patron = r'\b\d+[.,]\d+\b'
    not_answer = ["lo siento","no se", "no puedo", "no se menciona"]
    for i,r in enumerate(response["response"]):
        if any(word.lower() in r.lower() for word in not_answer):
            clean_response["response"][i] = " "
        else:
            result = re.findall(float_patron,r)
            clean_response["response"][i] = result
    return clean_response

In [ ]:
def upload_pdf(pdf_data):
    """
    Upload a PDF file, extract text, and save it to a text file.

    Args:
        pdf_data (file-like object): PDF file object.

    Returns:
        dict: Response indicating the success or failure of the operation.
    """
    try:

        path_txt = "data\\txt\\invoice.txt"

        # read PDF
        print(pdf_data)
        pdf_txt = extract_text_from_pdf(pdf_data)
        save_text_to_txt(pdf_txt, path_txt)

        return {'response': "Se ha subido correctamente el pdf"}
    except Exception as e:
        return {'error': f"Error al subir el pdf: {str(e)}"}

In [ ]:
def extract_link():
    """
    Extract a link (URL) from a PDF file.

    Returns:
        str or None: The extracted link or None if no link is found.
    """
    doc = fitz.open('data/pdf/invoice.pdf')

    for pages_num in range(doc.page_count):
        page = doc[pages_num]
        enlaces = page.get_links()

        for enlace in enlaces:
            url = enlace.get('uri')
            if url:
                return url
            else:
                return None

    doc.close()

In [ ]:
def extract_link_info(link):

    link_info = {}
    cups_pattern = re.compile(r'cups=[A-Z0-9]+')
    matches = cups_pattern.findall(link)

    cleaned_matches = [''.join(match.split()) for match in matches]

    link_info["cups20"] = cleaned_matches[0][5:]
    peak_regex = r'pP1=([0-9]+(?:\.[0-9]+)?)'
    link_info["peak_power"] = re.findall(peak_regex, link)[0].replace("pP1=", "")
    valley_regex = r'pP2=([0-9]+(?:\.[0-9]+)?)'
    link_info["valley_power"] = re.findall(valley_regex, link)[0].replace("pP2=", "")
    flat_regex = r'pP3=([0-9]+(?:\.[0-9]+)?)'
    if re.findall(flat_regex, link):
        link_info["flat_power"] = re.findall(flat_regex, link)[0].replace("pP3=", "")
    sd_regex = r'iniF=([0-9]{4}-[0-9]{2}-[0-9]{2})'
    link_info["start_date"] = re.findall(sd_regex, link)[0].replace("iniF=", "")
    ed_regex = r'finF=([0-9]{4}-[0-9]{2}-[0-9]{2})'
    link_info["end_date"] = re.findall(ed_regex, link)[0].replace("finF=", "") 
    id_regex = r'fFact=([0-9]{4}-[0-9]{2}-[0-9]{2})'
    link_info["invoice_date"] = re.findall(id_regex, link)[0].replace("fFact=", "")

    return link_info

In [ ]:
def extract_info_txt(link):
    """
    Extract the number of days and iva from a text file.

    Returns:
        dict: The extracted number of days or None if no match is found and iva or None.
    """
    info={}
    with open("data/txt/invoice.txt", 'r', encoding='utf-8') as file:
        file_txt = file.read()

    cups_pattern = re.compile(r'\b\d+\s*(?=\bdías\b)')
    info["days_invoice"] = cups_pattern.findall(file_txt)[0].replace(" ", "")
    percentage_pattern = r'\b(\d+(?:[.,]\d+)?)%\b'
    info["iva"] = re.findall(percentage_pattern, file_txt)[0].replace(",", ".")

    return info

In [ ]:
def extract_days():
    """
    Extract the number of days from a text file.

    Returns:
        str or None: The extracted number of days or None if no match is found.
    """
    with open("data/txt/invoice.txt", 'r', encoding='utf-8') as file:
        file_txt = file.read()
    cups_pattern = re.compile(r'\b\d+\s*(?=\bdías\b)')
    matches = cups_pattern.findall(file_txt)

    if matches:
        return matches[0].replace(" ", "")
    else:
        return None

In [ ]:
def image_to_text(res_img):
    """
    Convert an image to text using OCR and save the text to a text file.

    Args:
        res_img: Image data.

    Returns:
        dict: Response indicating the success or failure of the operation.
    """
    try:
        img_path = "data\\txt\\invoice.txt"
        save_txt = ""

        temp_name = f"temp_image.png"
        temp_path = os.path.join("data\\image", temp_name)
        res_img.save(temp_path)

        img = cv.imread(temp_path)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        reader = easyocr.Reader(['es'])
        img_txt = reader.readtext(gray)

        for n in img_txt:
            save_txt += n[1] + " "

        save_text_to_txt(save_txt, img_path)

        return {'response': "Se ha subido la imagen"}
    except Exception as e:
        return {'error': f"Error al subir la imagen: {str(e)}"}

In [ ]:
def prices_invoice():
    patron = re.compile(r'\b\d+\,\d{6}\b')
    with open("data/txt/invoice.txt", 'r', encoding='utf-8') as txt_file:
        txt_file = txt_file.read()

    matches = patron.findall(txt_file)

    cleaned_matches = [''.join(match.split()) for match in matches]

    patron = r'€/kWh|€/kW'
    measured = re.findall(patron, txt_file)
    return measured,cleaned_matches

def df_create(measured, cleaned_matches):
    data = {'precios': cleaned_matches, 'unidades': measured}
    df = pd.DataFrame.from_dict(data, orient='index').transpose()
    df.dropna(inplace=True)
    return df

In [ ]:
def assign_p_values(df):
    global p_counter_kW, p_counter_kWh
    
    if df['unidades'] == '€/kW':
        p_counter_kW += 1
        return f'p{p_counter_kW}'
    elif df['unidades'] == '€/kWh':
        p_counter_kWh += 1
        return f'p{p_counter_kWh}'
    else:
        return ''

In [ ]:
def json_prices(df):
    pdf_scarp_info = {
    "p1_price_kw": [],
    "p2_price_kw": [],
    "p3_price_kw": [],
    "p4_price_kw": [],
    "p5_price_kw": [],
    "p6_price_kw": [],
    "p1_price_kwh": [],
    "p2_price_kwh": [],
    "p3_price_kwh": [],
    "p4_price_kwh": [],
    "p5_price_kwh": [],
    "p6_price_kwh": [],
        }

    for index, row in df.iterrows():
        price_type = f"{row['P_values']}_price_{row['unidades'][2:].lower()}"
        pdf_scarp_info[price_type].append(row['precios'])
    
    return pdf_scarp_info

## 6. Modelo de Machine Learning

Desde Several Energy se propone la elaboración de un modelo de machine learning para establecer una predicción del precio del mercado de la luz en España. 

Este reto lo afrontamos haciendo primero una búsqueda del histórico de datos del precio de la luz. Finalmente recurrimos a www.omie.es de donde obtenemos un dataset con los precios por hora desde el 01/01/1998 hasta el 31/12/2023 , 25 años de datos listos para ser tratados.

Todo apuntaba a que podíamos enfocar el modelo de machine learning desde el punto de vista de una Serie temporal , empezamos a trababajar los datos para pasar de 24 feateures a 1 para usarla como serie , tratamos diferentes formas estadísticas de establecer esa única variable, viendo la desviación estandar de cada día, la media y precios máximos y minimos. Finalmente decidiamos que la media del precio diario seria la variable elegida para trabajar.

![Estacionalidad](../aux_temp/estacionalidad_mercado_diario.png)

#### Distribución de la serie temporal

![Distribución](../aux_temp/distribucion.png)

#### Gráfico de residuos

![Residuos](../aux_temp/2.png)

#### Gráfico de autocorrelación

![Autocorrelación](../aux_temp/autocorrelacion2.png)

#### Resultado de la predicción

![intento](../aux_temp/intento_35.png)

## 7. Reporte analítico

La creación de reporte analíco nos llevaba a ofrecer visualizaciones en tiempo real sobre las ventas realizadas y todos los KPIs (Key perfomance indicators) de ventas de los asesores. Nos enfrentabamos a que por razones de seguridad el cliente no podia darnos acceso al ERP para obtener acceso de los datos de ventas por lo que solo podríamos en el mejor de los casos tener datos de la parte previa a la venta, es decir el análisis del cliente, los datos de la factura,la propuesta y del asesor, para poder hacer un dashboard de visualizaciones y demostrar sus funcionalidades hemos creado un dataset falso. 

<iframe title="Dashboard Several Energy" width="1140" height="541.25" src="https://app.powerbi.com/reportEmbed?reportId=fa721417-ad80-425e-bc24-96b7a04ea0b1&autoAuth=true&ctid=bf86fbdb-f8c2-440e-923c-05a60dc2bc9b" frameborder="0" allowFullScreen="true"></iframe>

# 8.Librerías usadas e importaciones

## -----Web Development -----

In [ ]:
from flask import Flask, request, jsonify
from flask_limiter import Limiter
from flask_cors import CORS, cross_origin
from cerberus import Validator
from werkzeug.serving import make_server

## ----- General Utilities -----

In [ ]:
import os
from os import environ
import sys
import signal
import threading
from queue import Queue
import requests
from dotenv import load_dotenv

## ----- Custom Functions -----

In [ ]:
import functions

## ----- Computer Vision and OCR -----

In [ ]:
import cv2 as cv
import easyocr

## ----- Database Interaction -----

In [ ]:
import psycopg2
from sqlalchemy import create_engine

## ----- Web Scraping -----

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

## ----- Document Analysis -----

In [ ]:
import fitz
from langchain.chains import AnalyzeDocumentChain
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

## ----- Time Series Analysis -----

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima, ARIMA

## ----- Machine Learning and Data Analysis -----

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor, plot_tree
from scipy import stats

## ----- Data Visualization -----

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from IPython.display import IFrame

## ----- Streamlit and Data Handling -----

In [ ]:
import streamlit as st
import pandas as pd
import streamlit.components.v1 as c
import requests
import json
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os